In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import sys
import re
import json

from pyspark.ml import Pipeline
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol

### Implementacja przykładowego modułu do wczytywania danych
W przyszłości takie moduły powinny być pisane przez drugą grupę i załączane wraz z innymi modułami. W przyszłości ścieżki w takich modułach mogą być zaszyte w kodzie samego modułu. Lokalizacja danych wejściowych będzie także wskazywać na pliki zlokalizowane na HDFS, a nie w katalogu roboczym.

In [2]:
class DataLoaderTransformer(Transformer, HasInputCol, HasOutputCol):
    key = 'key'
    content = 'content'
    path = 'path'
    
    def __init__(self, sc, rootDir):
        self.sc = sc
        self.rootDir = rootDir
        super().__init__()

    def _transform(self, dataframe):
        def extractKey(entry):
            fpath, content = entry
            fname = fpath.split('/')[-1]
            key = re.sub('\.json$', '', fname)
            return (key, json.loads(content))
        
        mappings = self.sc.textFile(self.rootDir + 'mapping.csv')        \
            .map(lambda line: line.split('|'))                      \
            .toDF([self.path, self.key])
        
        statements = self.sc.wholeTextFiles(self.rootDir + 'statement/') \
            .map(extractKey)                                        \
            .toDF([self.key, self.content])
    
        return mappings.join(statements, 'key')

### Zaimportowanie modułów przygotowanych przez drugą grupę

In [3]:
from external.modules.posts import *

### Setup - uruchomienie Sparka
Domyślnie cały proces uruchamiany jest bez danych, pierwszy moduł w przepływie powinien być odpowiedzialny za ich wczytanie.

In [4]:
sconf = SparkConf()              \
    .setMaster('local[*]')       \
    .setAppName('PipelineFlow')

sc = SparkContext.getOrCreate(sconf)
sql = SQLContext(sc)
sess = SparkSession(sc)

df = sql.createDataFrame(sc.emptyRDD(), StructType([]))

### *Defnicja przepływu*
Przepływ definiowany jest poprzez stworzenie instancji kolejnych metod i przekazanie im kolumny (bądź kolumn) wejściowej i specyfikacja wyjścia każdej z metod. Następnie tworzony jest obiekt `pipeline` grupujący kolejne transformery w przepływ. Istotne jest aby zachować odpowiednią kolejność podczas przekazywania metod do pipeline, powinna ona być zgodna z zależnościami między metodami.

In [5]:
loader = DataLoaderTransformer(sc, '/home/jovyan/work/kascysko.blogspot.com/')

poster = PostTransformer()                  \
    .setInputCol(loader.content)            \
    .setOutputCol('posts')

translator = TranslateTransformer()         \
    .setInputCol(poster.getOutputCol())     \
    .setOutputCol('translated')
        
sentencer = SentenceTransformer()           \
    .setInputCol(translator.getOutputCol()) \
    .setOutputCol('sentences')              

pipeline = Pipeline(stages=[loader, poster, translator, sentencer])

### Uruchomienie przepływu i odczyt danych
Po dopasowaniu parametrów metod składających się na przepływ otrzymujemy model, który może być później zapisany dzieki czemu następnym razem nie będzie konieczności dobierania parametrów dla nowych danych.

In [6]:
pipelineModel = pipeline.fit(df)
output = pipelineModel.transform(df)

In [7]:
output.first().sentences[:5]

['I Jantar spiced with yeast beer :) described wcierka for me a bit too expensive',
 'I have banyan water with isany, but on holiday I am not very well with regularity.',
 'Jantar is waiting for autumn.',
 'I have this wcierkę on wishliscie but is piekielnie droga: /',
 'I like to use all wcierki :)']

### Zmiana przepływu

In [8]:
pos_counter = SpeechPartsTransformer()       \
    .setInputCol(translator.getOutputCol())  \
    .setOutputCol('pos_count')

pipeline_alt = Pipeline(stages=[loader, poster, translator, pos_counter])

In [9]:
out_alt = pipeline_alt.fit(df).transform(df)

In [10]:
out_alt.first().pos_count

{'CC': 28,
 'CD': 6,
 'DT': 69,
 'EX': 2,
 'IN': 106,
 'JJ': 57,
 'JJR': 3,
 'JJS': 2,
 'MD': 19,
 'NN': 137,
 'NNP': 23,
 'NNS': 32,
 'PDT': 1,
 'PRP': 106,
 'PRP$': 12,
 'RB': 66,
 'RBR': 3,
 'RBS': 1,
 'TO': 13,
 'UH': 3,
 'VB': 40,
 'VBD': 10,
 'VBG': 12,
 'VBN': 8,
 'VBP': 49,
 'VBZ': 33,
 'WP': 10,
 'WRB': 5}